In [1]:
# authorize google account to use bigquery
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

# authorize drive to pull model
from google.colab import drive
drive.mount('/content/drive')
]
# pull data for repo from bigquery
project_id = 'uic-capstone-int'
from google.cloud import bigquery

client = bigquery.Client(project=project_id)

Authenticated
Mounted at /content/drive


In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import random
import numpy as np
import datetime
from sklearn.metrics import r2_score

In [3]:
os.chdir('/content/drive/My Drive/Cloudbakers/Assets/')

In [19]:
from datetime import datetime
now=datetime.now()
today = now.strftime("%Y-%m-%d")
today

'2020-11-16'

In [10]:
WeeklyTS = pd.read_csv('BiggerDS-Part1.csv')
WeeklyTS['year'] = WeeklyTS['year'].astype(int)
WeeklyTS['month'] = WeeklyTS['month'].astype(int)
WeeklyTS['day'] = WeeklyTS['day'].astype(int)
WeeklyTS.drop('Unnamed: 0', axis=1, inplace=True)
WeeklyTS = WeeklyTS[WeeklyTS.year > 2014]
WeeklyTS['dateInt']=WeeklyTS['year'].astype(str) + WeeklyTS['month'].astype(str).str.zfill(2)+ WeeklyTS['day'].astype(str).str.zfill(2)
WeeklyTS['Date'] = pd.to_datetime(WeeklyTS['dateInt'], format='%Y%m%d')
WeeklyTS.drop(['year','month','day','dateInt'], axis=1, inplace=True)
x =pd.DataFrame(WeeklyTS.groupby('repoID').count())
x = x[x.Risk_Score>90].index.astype(int).tolist()
WeeklyTS = WeeklyTS[WeeklyTS.repoID.isin(x)]

In [11]:
WeeklyTS

,repoID,Risk_Score,Date
110,46566064.0,1.112586,2015-11-22
111,46566064.0,1.420722,2015-11-29
112,46566064.0,3.177162,2015-12-06
113,46566064.0,599.740572,2015-12-13
114,46566064.0,1188.086705,2015-12-20
...,...,...,...
72165,20287787.0,0.000000,2020-08-23
72166,20287787.0,0.000000,2020-08-30
72167,20287787.0,0.000000,2020-09-06
72168,20287787.0,0.000000,2020-09-13


In [7]:
#Concatenating smaller and larger dataset for training
SmallerDS = pd.read_csv('WeeklyTS.csv')
WeeklyTS['Date'] = WeeklyTS['Date'].dt.strftime('%Y-%m-%d')
type(WeeklyTS['Date'].iloc[0])
WeeklyTS = pd.concat([WeeklyTS,SmallerDS])
WeeklyTS.reset_index(level=0, inplace=True)
WeeklyTS.drop(['Unnamed: 0','index','level_0'], axis=1, inplace=True)


In [8]:
WeeklyTS

,index,repoID,Risk_Score,Date
0,110,46566064.0,1.112586,2015-11-22
1,111,46566064.0,1.420722,2015-11-29
2,112,46566064.0,3.177162,2015-12-06
3,113,46566064.0,599.740572,2015-12-13
4,114,46566064.0,1188.086705,2015-12-20
...,...,...,...,...
67362,72165,20287787.0,0.000000,2020-08-23
67363,72166,20287787.0,0.000000,2020-08-30
67364,72167,20287787.0,0.000000,2020-09-06
67365,72168,20287787.0,0.000000,2020-09-13


In [ ]:
def imputation(WeeklyTS,i):

    y = WeeklyTS[WeeklyTS.repoID==i].sort_values('Date')
    y['Date']= pd.to_datetime(y['Date'])
    y.index=y['Date']
    y.drop('Date',axis=1,inplace=True)
    idx = pd.date_range('2015-01-01', today, freq='W-SUN')
    y.index = pd.DatetimeIndex(y.index)
    y = y.reindex(idx, fill_value=0)

    y['Risk_Score']=y['Risk_Score'].replace({0:np.nan})
    y.drop('repoID', axis=1, inplace=True)
    y = y.assign(InterpolateLinear=y.Risk_Score.interpolate(method='linear', limit_direction='both'))
    y = y.assign(InterpolateTime=y.Risk_Score.interpolate(method='time', limit_direction='both'))
    y = y.assign(InterpolateQuadratic=y.Risk_Score.interpolate(method='quadratic',limit_direction='both'))
    y = y.assign(InterpolateCubic=y.Risk_Score.interpolate(method='cubic',limit_direction='both'))
    y = y.assign(InterpolateSLinear=y.Risk_Score.interpolate(method='slinear',limit_direction='both'))
    y = y.assign(InterpolateAkima=y.Risk_Score.interpolate(method='akima',limit_direction='both'))
    y = y.assign(InterpolatePoly5=y.Risk_Score.interpolate(method='polynomial', order=5, limit_direction='both')) 
    y = y.assign(InterpolatePoly7=y.Risk_Score.interpolate(method='polynomial', order=7,limit_direction='both'))
    y = y.assign(InterpolateSpline3=y.Risk_Score.interpolate(method='spline', order=3, limit_direction='both'))
    y = y.assign(InterpolateSpline4=y.Risk_Score.interpolate(method='spline', order=4,limit_direction='both'))
    y = y.assign(InterpolateSpline5=y.Risk_Score.interpolate(method='spline', order=5,limit_direction='both'))
    y = y.assign(FillMean=y.Risk_Score.fillna(y.Risk_Score.mean())) # imputing using the mean
    y = y.assign(FillMedian=y.Risk_Score.fillna(y.Risk_Score.median())) # imputing using the median
    y = y.assign(RollingMean=y.Risk_Score.fillna(y.Risk_Score.rolling(24,min_periods=1,).mean())) # imputing using the rolling mean
    y = y.assign(RollingMedian=y.Risk_Score.fillna(y.Risk_Score.rolling(24,min_periods=1,).median()))# imputing using the rolling median

    y.fillna(0, inplace=True)

    results = [(method, r2_score(y['Risk_Score'].fillna(0), y[method])) for method in list(y)[3:]]
    results_df = pd.DataFrame(np.array(results), columns=['Method', 'R_squared'])
    result_df = results_df.sort_values(by='R_squared', ascending=False)
    column_name =result_df.iloc[0].Method
    y=y[[column_name]]
    y.reset_index(level=0, inplace=True)
    y['Date'] = y['index'].dt.strftime('%Y-%m-%d')
    y.drop('index',axis=1, inplace=True)
    y.rename(columns={column_name:'Risk_Score'},inplace=True)
    y['repoID']=i
    return y


In [ ]:
repo_ids = WeeklyTS['repoID'].unique().tolist()
random.shuffle(repo_ids)

In [ ]:
WTS=pd.DataFrame()

for i in range(0,len(repo_ids)):
   
      impute = imputation(WeeklyTS,repo_ids[i])
      WTS=WTS.append(impute)

In [ ]:
WeeklyTS = WTS

In [ ]:
ScaledDS = pd.DataFrame()

for i in range(0,len(repo_ids)):
   TransformDS = WeeklyTS[WeeklyTS.repoID==repo_ids[i]]
   max = TransformDS['Risk_Score'].max()
   min = TransformDS['Risk_Score'].min()
   div = max - min
   scaled_risk_score = ((TransformDS.Risk_Score-min)/div)*10
   scaled_risk_score = pd.DataFrame(scaled_risk_score)
   scaled_risk_score.rename(columns={'Risk_Score':'Scaled_Risk_Score'}, inplace=True)
   scaledDS = pd.concat([TransformDS,scaled_risk_score],axis=1)
   ScaledDS = ScaledDS.append(scaledDS)
WeeklyTS = ScaledDS
WeeklyTS.drop('Risk_Score', axis=1, inplace=True)
WeeklyTS.rename(columns={'Scaled_Risk_Score':'Risk_Score'}, inplace=True)
WeeklyTS.reset_index(level=0,inplace=True)
WeeklyTS.drop('index',axis=1, inplace=True)

In [ ]:
# # Removing repositories with a lot of zeroes
# a = WeeklyTS.repoID.unique().tolist()
# m=WeeklyTS[WeeklyTS.Risk_Score!=0].groupby('repoID').count()
# repo_to_remove=m[m.Risk_Score<30].index.tolist()
# l3 = [x for x in a if x not in repo_to_remove]
# WeeklyTS = WeeklyTS[WeeklyTS.repoID.isin(l3)]
# WeeklyTS.reset_index(level=0, inplace=True)
# WeeklyTS

# Modeling 2

In [ ]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
from tensorflow import keras


## preprocession


In [ ]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler


def preprocessing(series):
    '''
    MinMax Scaling of the raw time series
    Args:
        series: the raw time series
    Returns:
        scaled_series and scaler object
    '''
    series = np.array(series)
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled = scaler.fit_transform(series.reshape(-1,1))
    scaled_series = scaled.reshape((len(series),))
    
    return scaled_series, scaler

def inverse_transform(series,scaler):
    '''
    Inverse transform of scales series
    Args:
        series: scaled series
        scaler: scaler object
    Returns:
        unscaled series
    '''
    return scaler.inverse_transform(series.reshape(-1,1))

def getSeries(data,p):
    '''
    Splits a given time series proportionally
    for training and testing purposes
    
    Args:
        data: numpy array or pandas series 
              containing the time series.
        p: float value that defines the 
           proportion of the series used
           for training.
    Returns:
        series: time series for training
        y_test: time series for testing
        n_test: number of timesteps 
                in the test series
    
    '''
    n = data.shape[0]
    n_train = int(n * p) 
    n_test = n - n_train

    x = np.arange(n)
        
    index_train = x[:n_train]
    index_test = x[n_train:]
    
    series = data[index_train]

    y_test = data[index_test]
    return series, y_test, n_test

def getInputOutput(series, input_size):
    '''
    Transforms the time series into desired 
    shape to be able to pass to the network
    
    Args:
        series: the time series.
        input_size: int that defines the length 
                    of the input sequence to be 
                    fed to the network
    Returns:
        X_train: input dataset
        y_train: output values
        X_test: the last available sequence
    
    '''
    
    series = np.array(series)
    xlen = len(series)
    xrows = xlen - input_size
    
    X_train, y_train = [], []
    
    for i in range(xrows):
        j = i + input_size
        a = series[i:j, np.newaxis]
        X_train.append(a)
        y_train.append(series[j])
    
    X_train,y_train = np.array(X_train), np.array(y_train)
    X_test = series[xrows:].reshape(1,input_size,1)
    
    
    return X_train, y_train, X_test

## Model

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
import itertools

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout




def build_LSTM(input_size,hidden_units, dropout, learning_rate):
    '''
    Builds the Network with LSTM hidden layers
    
    Args:
        input_size: int that defines the length 
                    of the input sequence to be 
                    fed to the network
        hidden_units: int/list specifying the number 
                      of hidden units in the hidden 
                      layer/layers
        dropout: boolean specifing whether to add dropout
                 with 0.5 rate per layer
        learning_rate: learning rate of the Adam 
                       optimization algorithm
    Returns:
        model: keras sequential model
        
    '''
    h = hidden_units
    

    model = Sequential()
    
    if isinstance(h,list):
    
        model.add(LSTM(h[0],
                   batch_input_shape=(1,input_size, 1), 
                   return_sequences=True, 
                   stateful=True))
                  
        if dropout:
            model.add(Dropout(rate=0.5))

        if len(h) > 2:
            #removing 1st and last units
            for index, units in enumerate(h[1:-1]):  
                model.add(LSTM(units, 
                               batch_input_shape=(1,h[index], 1), 
                               return_sequences=True, 
                               stateful=True)) 
                if dropout:
                    model.add(Dropout(rate=0.5))

        model.add(LSTM(h[-1], 
                       batch_input_shape=(1,h[-2], 1), 
                       return_sequences=False, 
                       stateful=True))
        if dropout:
            model.add(Dropout(rate=0.5))
    else:
        model.add(LSTM(h, 
                   batch_input_shape=(1,input_size, 1), 
                   return_sequences=False, 
                   stateful=True)) 
        if dropout:
            model.add(Dropout(rate=0.5))
        
    
    model.add(Dense(1))
    adam = keras.optimizers.Adam(lr=learning_rate)
    model.compile(loss='mse', optimizer=adam)
    return model

def predict_ahead(model,X_test,n_ahead):
    '''
    Makes predictions based on the last available sequence
    
    Args:
        model: keras sequential model
        X_test: the last available sequence
        n_ahead: number of predictions to make 
        
    Returns:
        predictions: numpy array containing the predicted values
        
    '''    
    predictions = np.zeros(n_ahead)
    predictions[0] = model.predict(X_test,batch_size = 1)
    
    if n_ahead > 1:
        for i in range(1,n_ahead):
            x_new = np.append(X_test[0][1:],predictions[i-1])
            X_test = x_new.reshape(1,x_new.shape[0],1)
            predictions[i] = model.predict(X_test,batch_size = 1)
    return predictions

def FitForecast(X_train, y_train, X_test, n_ahead, input_size,
                hidden_units, dropout, val_split, learning_rate, 
                epochs, trained_model):
    
    '''
    Fits a model and returns the predicted values. 
    Optionally weights from another network can be passed 
    
    Args:
        X_train: input dataset for training
        y_train: output dataset for training
        X_test: the last available sequence
        n_ahead: number of predictions to make 
        time_series: the time series of interest
        input_size: int that defines the length 
                    of the input sequence to be 
                    fed to the network
        hidden_units: int/list specifying the number 
                      of hidden units in the hidden 
                      layer/layers
        dropout: boolean specifing whether to add dropout
                 with 0.5 rate per layer
        learning_rate: learning rate of the Adam 
                       optimization algorithm
        epochs: int that defines the number of 
                training phases through the
                training dataset
        trained_model: already trained keras sequential 
                       model
        
    Returns:
        model: keras sequential model
        predictions: numpy array containing the predicted values
        history: training and validation loss history
        
    '''
    model = build_LSTM(input_size,hidden_units,dropout, learning_rate)
    
    if trained_model is not None:
        model.set_weights(weights = trained_model.get_weights())        
    
    
    history = model.fit(x=X_train, y=y_train, 
                batch_size=1, epochs=epochs, 
                verbose=1, validation_split=val_split,
                shuffle=False)

    predictions = predict_ahead(model,X_test,n_ahead)
    return model, predictions, history

def FitEvaluate(time_series,params):
    '''
    Calls the pipeline to fit an LSTM model to the 
    given time series
    
    Args:
        time_series: the time series of interest
        params: a dictionary specifying parameters
                {input_size, hidden_units, dropout,
                learning_rate, n_ahead, val_split, 
                epochs, verbose, plot}
    Returns:
        model: keras sequential model      
        mse: mean squared error of the prediction
        history: training and validation loss history
        
    '''   
    
    for k in params.keys():
        globals()[k] = params[k]
    
    
    scaled_series, scaler = preprocessing(time_series)
    series, y_test, n_test = getSeries(scaled_series,0.8)
    X_train,y_train,X_test = getInputOutput(series,input_size)
    
    # show only n_ahead number of actual values
    y_test = y_test[np.arange(n_ahead)]

    new_model, predictions, history = FitForecast(X_train,y_train,X_test,n_ahead,
                                        input_size,hidden_units,dropout, val_split,
                                        learning_rate,epochs,trained_model=None)
    
    # rescaling
    series = inverse_transform(series, scaler)
    y_test = inverse_transform(y_test, scaler)
    predictions = inverse_transform(predictions, scaler)
    
    mse = mean_squared_error(y_true=y_test,y_pred=predictions)
    
    if verbose:
        print('\n')
        print('======== Prediction Evaluation =========')
        print('MSE is {}'.format(round(mse,4)))
        
    if plot:
        ViewLoss(history)
        view_predictions(series,predictions,y_test,'Actual vs Forecast')
    return new_model, mse, history, predictions, y_test

def TransferLearning(time_series,params,model):
    '''
    Calls the pipeline to fit an LSTM model to the 
    given time series with and without knowledge
    transfer
    
    Args:
        time_series: the time series of interest
        params: a dictionary specifying parameters
                {input_size, hidden_units, dropout,
                learning_rate, n_ahead, val_split, 
                epochs, verbose, plot}
        model: already trained keras sequential 
               model
    Returns:
        mean squared errors of the predictions
        and 2 plots         
    '''
    for k in params.keys():
        globals()[k] = params[k]
    
    val_split = 0
    
    scaled_series, scaler = preprocessing(time_series)
    series, y_test, n_test = getSeries(scaled_series,0.8)
    X_train,y_train,X_test = getInputOutput(series,input_size)

    # show only n_ahead number of actual values
    y_test = y_test[np.arange(n_ahead)]

    
    print('*** Fitting a model without knowledge transfer ***')
    model_noTransfer, predictions_noTransfer, _ = FitForecast(X_train,y_train,
                                                             X_test,n_ahead,
                                               input_size,hidden_units,
                                                             dropout,val_split,
                                                             learning_rate,
                                               epochs,
                                                             trained_model=None)
    
    print('\n')
    print('*** Fitting a model with knowledge transfer ***')
    model_withTransfer, predictions_withTransfer, _ = FitForecast(X_train,y_train,
                                                                 X_test,n_ahead,
                                                                 input_size,hidden_units,
                                                                 dropout,val_split,
                                                                 learning_rate,
                                                                 epochs,
                                                                 trained_model=model)
    
    # rescaling
    series = inverse_transform(series, scaler)
    y_test = inverse_transform(y_test, scaler)
    predictions_noTransfer = inverse_transform(predictions_noTransfer, scaler)
    predictions_withTransfer = inverse_transform(predictions_withTransfer, scaler)
    
    mse_noTransfer = mean_squared_error(y_true=y_test,y_pred=predictions_noTransfer)
    mse_withTransfer = mean_squared_error(y_true=y_test,y_pred=predictions_withTransfer)
      
    print('\n')
    print('======== Results for no knowledge transfer =========')
    print('The RMSE is {}'.format(round(np.sqrt(mse_noTransfer),4)))
    print('\n')
    print('======== Results for knowledge transfer =========')
    print('The RMSE is {}'.format(round(np.sqrt(mse_withTransfer),4)))
        
    view_predictions(series,predictions_noTransfer,y_test,title='Without Transfer')
    view_predictions(series,predictions_withTransfer,y_test,title='With Transfer')

    return model_withTransfer

def GridSearch(series,params_grid):
    '''
    Runs a grid search over specified parameter ranges
    Args: 
        series: the time series of interest
        params_grid: a dictionary specifying parameters
                    {input_size, hidden_units, dropout,
                    learning_rate, n_ahead, val_split, 
                    epochs, verbose, plot} and their 
                    possible value ranges
    Returns:
        model: the model with the lowest MSE 
        logs: logs of all combinations        
       
    '''
    param_names = list(params_grid.keys())
    param_values = list(params_grid.values()) 
    combinations = list(itertools.product(*param_values))
    
    logs = pd.DataFrame(combinations,columns=param_names)
    
    mse_prev = 50
    for index, comb in enumerate(combinations):
        
        print('Fitting {}/{} model'.format(index+1,len(combinations)))
        params = dict(zip(param_names,comb))
        model, mse, history, predn, y_test = FitEvaluate(series,params)
        train_loss = history.history['loss']
        val_loss = history.history['val_loss']
        
        
        if mse < mse_prev:
            mse_prev = mse
            best_model = model
            best_predictions = predn
            yt = y_test
        
        logs.at[index,'mse'] = mse
        logs.at[index,'mean_training_loss'] = np.mean(train_loss)
        logs.at[index,'std_training_loss'] = np.std(train_loss)
        logs.at[index,'mean_val_loss'] = np.mean(val_loss)
        logs.at[index,'std_val_loss'] = np.std(val_loss)
    logs.to_csv('results.csv',index=False)
    # view_predictions(series,best_predictions,y_test,'Actual vs Forecast')
    return best_model, logs, best_predictions, yt


def generalTuning(series1, series2, params):
    
    '''
    Fitting 3 models trained on 
    {general domain,general+in-domain,in-domain only}
    and comparing them
    Args: 
        series1: target related time series 
        series2: target time series
        params: a dictionary specifying parameters
                {input_size, hidden_units, dropout,
                learning_rate, n_ahead, val_split, 
                epochs, verbose, plot}
    Returns:
        mean squared errors of the predictions
        and 3 plots 
    
    '''
    
    time_series = np.concatenate([series1,series2])
        
    
    
    for k in params.keys():
        globals()[k] = params[k]
        
    val_split = 0
    
    # preprocessing general series
    scaled_general, scaler_general = preprocessing(time_series)
    series_general, _, __ = getSeries(scaled_general,0.9)
    X_train_general,y_train_general,___ = getInputOutput(series_general,input_size)

    # preprocessing the target series
    scaled_target, scaler_target = preprocessing(series2) 
    series_target, y_test_target, n_test = getSeries(scaled_target,0.8)
    X_train_target,y_train_target,X_test_target = getInputOutput(series_target,input_size)

    # comparing predictions with only n_ahead number of actual values
    y_test_target = y_test_target[np.arange(n_ahead)]

    
    # build and train a model on the general domain (time_series)
    
    print('*** Fitting a model on general domain ***')

    
    model_general, predictions_pre_tuned, hist = FitForecast(X_train_general,y_train_general,
                                                             X_test_target,n_ahead,
                                                             input_size, hidden_units,
                                                             dropout, val_split,
                                                             learning_rate,
                                                             epochs,
                                                             trained_model=None)                                               
                                                             
                                                             
                                                        
 
    # initiallize a model for target
    model_tuned = build_LSTM(input_size, hidden_units, dropout, learning_rate)
        
    # transfer the knowledge from the pre-trained model
    # and tune it only on the target domain (series2)
    model_tuned.set_weights(weights=model_general.get_weights())        
    
    
    print('\n *** Tuning a model on target domain ***')

    model_tuned.fit(x=X_train_target, y=y_train_target, 
                        batch_size=1, epochs=epochs, 
                        verbose=1, validation_data=None,
                        shuffle=False)
    
    predictions_tuned = predict_ahead(model_tuned,X_test_target,n_ahead)
    
    
   
    print('\n *** Fitting a model on target domain only ***')

    model_target, predictions_target,hist2 = FitForecast(X_train_target,y_train_target,
                                                   X_test_target,n_ahead,
                                                   input_size, hidden_units,
                                                   dropout, val_split,
                                                   learning_rate,
                                                   2 * epochs,
                                                   trained_model=None)                                               


    series_target = inverse_transform(series_target,scaler_target)
    y_test_target = inverse_transform(y_test_target,scaler_target)
    
    series_general = inverse_transform(series_general,scaler_general)
    predictions_pre_tuned = inverse_transform(predictions_pre_tuned,scaler_target)
    mse_pre_tuned = mean_squared_error(y_true=y_test_target,y_pred=predictions_pre_tuned)
    
    predictions_tuned = inverse_transform(predictions_tuned,scaler_target)
    mse_tuned = mean_squared_error(y_true=y_test_target,y_pred=predictions_tuned)
    
    predictions_target = inverse_transform(predictions_target,scaler_target)  
    mse_target = mean_squared_error(y_true=y_test_target,y_pred=predictions_target)
    
    print('\n')
    print('======== Results for pre_tuned model =========')
    print('The RMSE is {}'.format(round(np.sqrt(mse_pre_tuned),4)))
    print('\n')
    print('======== Results for tuned model =========')
    print('The RMSE is {}'.format(round(np.sqrt(mse_tuned),4)))
    
    print('\n')
    print('======== Results for target model only =========')
    print('The RMSE is {}'.format(round(np.sqrt(mse_target),4)))
    
        
    view_predictions([series_general,series_target],predictions_pre_tuned,y_test_target,title='Pre-tuned')
    view_predictions([series_general,series_target],predictions_tuned,y_test_target,title='Tuned')
    view_predictions(series_target,predictions_target,y_test_target,title='Target only')


## Visualization

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

def ACF(series,lags):
    '''
    Plots the Autocorrelation graph
    of the series
    Args:
        series: time series of interest
        lags: int specifying number of 
              periods to look back
    Returns: plot
    '''
    plot_acf(series, lags=lags)
    plt.savefig('ACF.png')
    plt.show()

def plotSeries(series_dict):
    '''
    Plots several time series
    Args:
        series_dict: dict with series names as keys
                     and time series as values
    Returns:
        plot
        
    '''
    n_series = len(series_dict) - 1
    names = list(series_dict.keys())
    
    plt.figure(figsize=(10,20))
    
    for i in range(n_series):
        plt.subplot(n_series,1,i+1)
        plt.title(names[i+1])
        plt.plot(series_dict['t'],series_dict[names[i+1]])
        
    plt.show()

def ViewLoss(history):
    '''
    Plots the history of model training
    '''
    plt.plot(history.history['loss'],label='Train')
    plt.plot(history.history['val_loss'],label='Val')
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    plt.savefig('loss-history.png')
    plt.show()

def view_predictions(series,predictions,actual,title):
    '''
    Plots the results of the predictions made by the model
    
    Args:
        series: the time series used for training the network
        predictions: numpy array containing the predicted values
        actual: the actual time series not seen by the network
        title: the title of the plot
        
    Returns:
        plot        
        
    '''  
    
    plt.figure(figsize=(8,4))
    plt.title(title)
    
    if isinstance(series,list):
        train_index = np.arange(len(series[0]))
        test_index = len(series[0]) + np.arange(len(actual))
        
        plt.plot(train_index,series[0], label = 'general')
        
    else:
        train_index = np.arange(len(series))
        test_index = len(series) + np.arange(len(actual))        
        plt.plot(train_index,series,label = 'training')

    if len(predictions) > 4:
        plt.plot(test_index,predictions,label = 'prediction',color='g')
        plt.plot(test_index,actual,label = 'actual',color='orange')
    else:
        plt.scatter(test_index,predictions,label = 'prediction',color='g')
        plt.scatter(test_index,actual,label = 'actual',color='orange')    
    
    plt.xlabel('Index')
    plt.ylabel('Data')
    
    plt.legend(loc='upper left')
    plt.savefig('{}_{}.png'.format(title,len(series)))
    plt.show()

## Grid Search

In [ ]:
WeeklyTS = pd.read_csv('/content/drive/My Drive/Cloudbakers/Assets/WeeklyTS.csv')
WeeklyTS.drop('Unnamed: 0', axis=1, inplace=True)
WeeklyTS = WeeklyTS[WeeklyTS.year > 2014]
WeeklyTS['dateInt']=WeeklyTS['year'].astype(str) + WeeklyTS['month'].astype(str).str.zfill(2)+ WeeklyTS['day'].astype(str).str.zfill(2)
WeeklyTS['Date'] = pd.to_datetime(WeeklyTS['dateInt'], format='%Y%m%d')
WeeklyTS.drop(['year','month','day','dateInt'], axis=1, inplace=True)

In [ ]:
now=datetime.now()
today = now.strftime("%Y-%m-%d")
today

'2020-11-06'

In [ ]:
repo_ids = WeeklyTS['repoID'].unique().tolist()

In [ ]:
Sub_WeeklyTS = WeeklyTS[WeeklyTS['repoID']==23122983].sort_values(by=['Date'])
Sub_WeeklyTS.index=Sub_WeeklyTS['Date']
Sub_WeeklyTS.drop('Date',axis=1,inplace=True)
idx = pd.date_range('2015-01-01', today, freq='W-SUN')
Sub_WeeklyTS.index = pd.DatetimeIndex(Sub_WeeklyTS.index)
Sub_WeeklyTS = Sub_WeeklyTS.reindex(idx, fill_value=0)
Sub_WeeklyTS.drop(['repoID'], axis=1, inplace=True)

In [ ]:
# date = Sub_WeeklyTS.index.strftime("%Y-%m-%d").to_numpy()
# # type(date)
Risk_Score = Sub_WeeklyTS.Risk_Score.to_numpy()

In [ ]:
# Sub_Weekly_dict = {'t':date,
#               'Risk_Score':Risk_Score}
# plotSeries(Sub_Weekly_dict)

In [ ]:
Risk_Score

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        1.30712982,  4.7714807 ,  3.92138947,  4.8710617 ,  3.74266067,
        2.15722105,  9.6425424 ,  1.30712982,  0.        ,  6.53564912,
        0.        ,  4.3144421 ,  2.61425965,  4.7714807 ,  2.61425965,
        1.30712982,  2.15722105,  3.46435088,  2.43553085,  6.07861053,
        0.85009123,  2.61425965,  0.        ,  2.15722105,  1.30712982,
        2.61425965,  0.        ,  3.92138947,  0.85009123,  2.61425965,
        0.        ,  1.30712982,  2.15722105,  3.92138947,  0.85009123,
        2.15722105,  3.46435088,  3.46435088,  3.46435088,  1.30712982,
        3.92138947,  2.15722105,  0.        ,  3.92138947,  1.47831409,
        2.61425965,  2.61425965,  1.30712982,  2.61425965,  3.28562208,
        5.0497905 ,  1.30712982,  0.        ,  2.15722105,  4.7714807 ,
        7.20701155,  4.7714807 ,  4.3144421 ,  6.07861053,  3.46435088,
        3.46435088,  2.61425965,  1.30712982,  2.61425965,  1.30

In [ ]:
# to perform a grid search over the parameter
params_grid = {'input_size': [3,10,15,30,70],
              'hidden_units':[100,[100,50],[100,50,50]],
              'dropout': [True, False],
              'learning_rate':[4e-5],
              'n_ahead':[5],
              'val_split': [0.2],
              'epochs':[20],
              'verbose':[False],
              'plot':[False]}

# Risk_Score = Sub_WeeklyTS['Risk_Score'].to_numpy()
model_, logs = GridSearch(Risk_Score,params_grid)

Fitting 1/30 model
Epoch 1/20
192/192 [==============================] - 1s 4ms/step - loss: 0.0632 - val_loss: 0.0175
Epoch 2/20
192/192 [==============================] - 0s 2ms/step - loss: 0.0450 - val_loss: 0.0155
Epoch 3/20
192/192 [==============================] - 0s 2ms/step - loss: 0.0407 - val_loss: 0.0160
Epoch 4/20
192/192 [==============================] - 0s 2ms/step - loss: 0.0408 - val_loss: 0.0162
Epoch 5/20
192/192 [==============================] - 0s 2ms/step - loss: 0.0416 - val_loss: 0.0161
Epoch 6/20
192/192 [==============================] - 0s 2ms/step - loss: 0.0384 - val_loss: 0.0163
Epoch 7/20
192/192 [==============================] - 0s 2ms/step - loss: 0.0397 - val_loss: 0.0163
Epoch 8/20
192/192 [==============================] - 0s 2ms/step - loss: 0.0388 - val_loss: 0.0163
Epoch 9/20
192/192 [==============================] - 0s 2ms/step - loss: 0.0376 - val_loss: 0.0165
Epoch 10/20
192/192 [==============================] - 0s 3ms/step - loss: 0.0394

ValueError: ignored

In [ ]:
# model with lowest MSE
params = {'input_size': 10,
          'hidden_units':100,
          'dropout': True,
          'learning_rate':4e-5,
          'n_ahead':5,
          'val_split': 0.2,
          'epochs':20,
          'verbose':False,
          'plot':False}

In [ ]:
model_, mse, hist, best_prediction, yt  = FitEvaluate(Risk_Score,params)

Epoch 1/20
187/187 [==============================] - 1s 6ms/step - loss: 0.0738 - val_loss: 0.0174
Epoch 2/20
187/187 [==============================] - 1s 4ms/step - loss: 0.0481 - val_loss: 0.0150
Epoch 3/20
187/187 [==============================] - 1s 4ms/step - loss: 0.0376 - val_loss: 0.0157
Epoch 4/20
187/187 [==============================] - 1s 4ms/step - loss: 0.0397 - val_loss: 0.0157
Epoch 5/20
187/187 [==============================] - 1s 4ms/step - loss: 0.0390 - val_loss: 0.0159
Epoch 6/20
187/187 [==============================] - 1s 4ms/step - loss: 0.0392 - val_loss: 0.0158
Epoch 7/20
187/187 [==============================] - 1s 4ms/step - loss: 0.0379 - val_loss: 0.0159
Epoch 8/20
187/187 [==============================] - 1s 4ms/step - loss: 0.0383 - val_loss: 0.0160
Epoch 9/20
187/187 [==============================] - 1s 4ms/step - loss: 0.0378 - val_loss: 0.0160
Epoch 10/20
187/187 [==============================] - 1s 4ms/step - loss: 0.0396 - val_loss: 0.0160

In [ ]:
ViewLoss(hist)
view_predictions(Risk_Score,best_prediction,yt,'Actual vs Forecast')
print('mean-squared-error: {}'.format(mse))

## Transfer Learning

In [ ]:
Sub_WeeklyTS = WeeklyTS[WeeklyTS['repoID']==163683035.0].sort_values(by=['Date'], ascending=True)
Sub_WeeklyTS.index=Sub_WeeklyTS['Date']
Sub_WeeklyTS.drop(['Date','repoID'], axis=1, inplace=True)
Risk_Score = Sub_WeeklyTS.Risk_Score.to_numpy()

In [ ]:

# model_ = keras.models.load_model('Transfer_learning_model')


# model with lowest MSE
params = {'input_size': 10,
          'hidden_units':100,
          'dropout': True,
          'learning_rate':4e-5,
          'n_ahead':5,
          'val_split': 0.2,
          'epochs':20,
          'verbose':False,
          'plot':False}
model_, mse, hist, best_prediction, yt  = FitEvaluate(Risk_Score,params)

Epoch 1/20
187/187 [==============================] - 1s 5ms/step - loss: 0.0000e+00 - val_loss: 0.5181
Epoch 2/20
187/187 [==============================] - 1s 4ms/step - loss: 6.2822e-07 - val_loss: 0.5351
Epoch 3/20
187/187 [==============================] - 1s 3ms/step - loss: 4.9163e-06 - val_loss: 0.5132
Epoch 4/20
187/187 [==============================] - 1s 3ms/step - loss: 3.1239e-07 - val_loss: 0.5084
Epoch 5/20
187/187 [==============================] - 1s 4ms/step - loss: 1.2966e-07 - val_loss: 0.5146
Epoch 6/20
187/187 [==============================] - 1s 3ms/step - loss: 1.9476e-07 - val_loss: 0.5171
Epoch 7/20
187/187 [==============================] - 1s 3ms/step - loss: 1.4500e-06 - val_loss: 0.5001
Epoch 8/20
187/187 [==============================] - 1s 3ms/step - loss: 2.0325e-06 - val_loss: 0.5233
Epoch 9/20
187/187 [==============================] - 1s 3ms/step - loss: 7.1029e-07 - val_loss: 0.5101
Epoch 10/20
187/187 [==============================] - 1s 3ms/st

In [ ]:
for r in range(0,len(repo_ids)):
        print('******Training on Repo ID: {} ********'.format(repo_ids[r]))

        Sub_WeeklyTS = WeeklyTS[WeeklyTS['repoID']==repo_ids[r]].sort_values(by=['Date'])
        Sub_WeeklyTS.index=Sub_WeeklyTS['Date']
        Sub_WeeklyTS.drop('Date',axis=1,inplace=True)
        Sub_WeeklyTS.drop(['repoID'], axis=1, inplace=True)
        Risk_Score = Sub_WeeklyTS.Risk_Score.to_numpy()


        model_ = TransferLearning( Risk_Score,params,model=model_)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# serialize model to JSON
model_json =model_.to_json()

with open("Transfer_learning_model_BiggerDS_Imputation.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model_.save('Transfer_learning_model_BiggerDS_Imputation.h5')

model_.save_weights("Transfer_learning_model_BiggerDS_Imputation.h5")

In [ ]:
model_.get_weights()

# Testing model

In [ ]:
def Test_FitEvaluate(time_series,params,model_):
    '''
    Calls the pipeline to fit an LSTM model to the 
    given time series
    
    Args:
        time_series: the time series of interest
        params: a dictionary specifying parameters
                {input_size, hidden_units, dropout,
                learning_rate, n_ahead, val_split, 
                epochs, verbose, plot}
    Returns:
        model: keras sequential model      
        mse: mean squared error of the prediction
        history: training and validation loss history
        
    '''   
    
    for k in params.keys():
        globals()[k] = params[k]
    
    
    scaled_series, scaler = preprocessing(time_series)
    series, y_test, n_test = getSeries(scaled_series,0.8)
    X_train,y_train,X_test = getInputOutput(series,input_size)
    
    # show only n_ahead number of actual values
    y_test = y_test[np.arange(n_ahead)]

    new_model, predictions, history = FitForecast(X_train,y_train,X_test,n_ahead,
                                        input_size,hidden_units,dropout, val_split,
                                        learning_rate,epochs,trained_model=model_)
    
    # rescaling
    series = inverse_transform(series, scaler)
    y_test = inverse_transform(y_test, scaler)
    predictions = inverse_transform(predictions, scaler)
    
    mse = mean_squared_error(y_true=y_test,y_pred=predictions)
    
    print('MSE is {}'.format(round(mse,4)))
        

    ViewLoss(history)
    view_predictions(series,predictions,y_test,'Actual vs Forecast')
    return new_model, mse, history, predictions, y_test

In [ ]:
from tensorflow import keras

model_ = keras.models.load_model('Transfer_learning_model')

In [ ]:
Sub_WeeklyTS = WeeklyTS[WeeklyTS['repoID']==20873044].sort_values(by=['Date'])
Sub_WeeklyTS.index=Sub_WeeklyTS['Date']
Sub_WeeklyTS.drop('Date',axis=1,inplace=True)
idx = pd.date_range('2015-01-01', today, freq='W-SUN')
Sub_WeeklyTS.index = pd.DatetimeIndex(Sub_WeeklyTS.index)
Sub_WeeklyTS = Sub_WeeklyTS.reindex(idx, fill_value=0)
Sub_WeeklyTS.drop(['repoID'], axis=1, inplace=True)
Risk_Score = Sub_WeeklyTS.Risk_Score

In [ ]:
new_model_, mse, history, predictions, y_test = Test_FitEvaluate(Risk_Score, params, model_)